# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Feb 15 2023 9:12AM,257422,19,9132418,Eywa Pharma Inc.,Released
1,Feb 15 2023 9:12AM,257422,19,9132419,Eywa Pharma Inc.,Released
2,Feb 15 2023 9:12AM,257422,19,9132429,Eywa Pharma Inc.,Released
3,Feb 15 2023 9:04AM,257421,10,MSP218475,"Methapharm, Inc.",Released
4,Feb 15 2023 9:00AM,257420,10,MSP218551,"Methapharm, Inc.",Released
5,Feb 15 2023 9:00AM,257420,10,MSP218553,"Methapharm, Inc.",Released
6,Feb 15 2023 9:00AM,257420,10,MSP218554,"Methapharm, Inc.",Released
7,Feb 15 2023 9:00AM,257420,10,MSP218555,"Methapharm, Inc.",Released
8,Feb 15 2023 9:00AM,257420,10,MSP218412,"Methapharm, Inc.",Released
9,Feb 15 2023 9:00AM,257420,10,MSP218558,"Methapharm, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
46,257418,Released,1
47,257419,Released,3
48,257420,Released,10
49,257421,Released,1
50,257422,Released,3


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
257418,NaN,NaN,1.0
257419,NaN,NaN,3.0
257420,NaN,NaN,10.0
257421,NaN,NaN,1.0
257422,NaN,NaN,3.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
257321,0.0,0.0,1.0
257336,0.0,1.0,0.0
257339,0.0,1.0,0.0
257340,28.0,9.0,7.0
257341,0.0,0.0,2.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
257321,0,0,1
257336,0,1,0
257339,0,1,0
257340,28,9,7
257341,0,0,2


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,257321,0,0,1
1,257336,0,1,0
2,257339,0,1,0
3,257340,28,9,7
4,257341,0,0,2


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,257321,,,1
1,257336,,1,
2,257339,,1,
3,257340,28,9,7
4,257341,,,2


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Feb 15 2023 9:12AM,257422,19,Eywa Pharma Inc.
3,Feb 15 2023 9:04AM,257421,10,"Methapharm, Inc."
4,Feb 15 2023 9:00AM,257420,10,"Methapharm, Inc."
14,Feb 15 2023 8:52AM,257419,10,Emerginnova
17,Feb 15 2023 8:51AM,257418,12,SugarBear
18,Feb 15 2023 8:49AM,257417,10,ISDIN Corporation
43,Feb 15 2023 8:48AM,257416,12,Hush Hush
64,Feb 15 2023 8:46AM,257415,10,ISDIN Corporation
83,Feb 15 2023 8:30AM,257414,16,"SHL Pharma, LLC"
84,Feb 15 2023 8:30AM,257392,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Feb 15 2023 9:12AM,257422,19,Eywa Pharma Inc.,,,3
1,Feb 15 2023 9:04AM,257421,10,"Methapharm, Inc.",,,1
2,Feb 15 2023 9:00AM,257420,10,"Methapharm, Inc.",,,10
3,Feb 15 2023 8:52AM,257419,10,Emerginnova,,,3
4,Feb 15 2023 8:51AM,257418,12,SugarBear,,,1
5,Feb 15 2023 8:49AM,257417,10,ISDIN Corporation,,,25
6,Feb 15 2023 8:48AM,257416,12,Hush Hush,,,21
7,Feb 15 2023 8:46AM,257415,10,ISDIN Corporation,,,19
8,Feb 15 2023 8:30AM,257414,16,"SHL Pharma, LLC",,,1
9,Feb 15 2023 8:30AM,257392,10,ISDIN Corporation,,,5


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 15 2023 9:12AM,257422,19,Eywa Pharma Inc.,3,,
1,Feb 15 2023 9:04AM,257421,10,"Methapharm, Inc.",1,,
2,Feb 15 2023 9:00AM,257420,10,"Methapharm, Inc.",10,,
3,Feb 15 2023 8:52AM,257419,10,Emerginnova,3,,
4,Feb 15 2023 8:51AM,257418,12,SugarBear,1,,
5,Feb 15 2023 8:49AM,257417,10,ISDIN Corporation,25,,
6,Feb 15 2023 8:48AM,257416,12,Hush Hush,21,,
7,Feb 15 2023 8:46AM,257415,10,ISDIN Corporation,19,,
8,Feb 15 2023 8:30AM,257414,16,"SHL Pharma, LLC",1,,
9,Feb 15 2023 8:30AM,257392,10,ISDIN Corporation,5,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 15 2023 9:12AM,257422,19,Eywa Pharma Inc.,3,,
1,Feb 15 2023 9:04AM,257421,10,"Methapharm, Inc.",1,,
2,Feb 15 2023 9:00AM,257420,10,"Methapharm, Inc.",10,,
3,Feb 15 2023 8:52AM,257419,10,Emerginnova,3,,
4,Feb 15 2023 8:51AM,257418,12,SugarBear,1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 15 2023 9:12AM,257422,19,Eywa Pharma Inc.,3.0,NaN,NaN
1,Feb 15 2023 9:04AM,257421,10,"Methapharm, Inc.",1.0,NaN,NaN
2,Feb 15 2023 9:00AM,257420,10,"Methapharm, Inc.",10.0,NaN,NaN
3,Feb 15 2023 8:52AM,257419,10,Emerginnova,3.0,NaN,NaN
4,Feb 15 2023 8:51AM,257418,12,SugarBear,1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 15 2023 9:12AM,257422,19,Eywa Pharma Inc.,3.0,0.0,0.0
1,Feb 15 2023 9:04AM,257421,10,"Methapharm, Inc.",1.0,0.0,0.0
2,Feb 15 2023 9:00AM,257420,10,"Methapharm, Inc.",10.0,0.0,0.0
3,Feb 15 2023 8:52AM,257419,10,Emerginnova,3.0,0.0,0.0
4,Feb 15 2023 8:51AM,257418,12,SugarBear,1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,7978654,297.0,18.0,28.0
12,772240,22.0,1.0,0.0
15,257354,6.0,0.0,1.0
16,257414,1.0,0.0,0.0
19,1801760,9.0,1.0,0.0
20,257336,0.0,1.0,0.0
22,772178,2.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,7978654,297.0,18.0,28.0
1,12,772240,22.0,1.0,0.0
2,15,257354,6.0,0.0,1.0
3,16,257414,1.0,0.0,0.0
4,19,1801760,9.0,1.0,0.0
5,20,257336,0.0,1.0,0.0
6,22,772178,2.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,297.0,18.0,28.0
1,12,22.0,1.0,0.0
2,15,6.0,0.0,1.0
3,16,1.0,0.0,0.0
4,19,9.0,1.0,0.0
5,20,0.0,1.0,0.0
6,22,2.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,297.0
1,12,Released,22.0
2,15,Released,6.0
3,16,Released,1.0
4,19,Released,9.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20,22
Status,,,,,,,
Completed,28.0,0.0,1.0,0.0,0.0,0.0,0.0
Executing,18.0,1.0,0.0,0.0,1.0,1.0,1.0
Released,297.0,22.0,6.0,1.0,9.0,0.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20,22
0,Completed,28.0,0.0,1.0,0.0,0.0,0.0,0.0
1,Executing,18.0,1.0,0.0,0.0,1.0,1.0,1.0
2,Released,297.0,22.0,6.0,1.0,9.0,0.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20,22
0,Completed,28.0,0.0,1.0,0.0,0.0,0.0,0.0
1,Executing,18.0,1.0,0.0,0.0,1.0,1.0,1.0
2,Released,297.0,22.0,6.0,1.0,9.0,0.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()